# Importing required modules and packages 

In [328]:
!conda install -c conda-forge geopy --yes; # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [329]:
!conda install -c conda-forge geocoder --yes;

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [330]:
!conda install -c conda-forge folium=0.5.0 --yes;

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [331]:
#from zipfile import ZipFile
from sklearn.cluster import KMeans # import k-means from clustering stage
import folium
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
import io
import matplotlib.cm as cm
import matplotlib.colors as colors
import json

<div class="alert alert-block alert-danger">
<b> Skip the following lines until next red box:</b> Data is already downloaded, parsed and I already took a sample
</div>

## Downloading zip file and extracting it

In [17]:
#!wget -q -O 'zipsing' http://download.geonames.org/export/zip/SG.zip

In [18]:
#with ZipFile('zipsing', 'r') as zip:
#    zip.extract('SG.txt')

## Parsing the txt file and renaming columns / You can pass directly to the saved CSV file

In [89]:
data = pd.read_csv('SG.txt',sep='	', header=None)

In [326]:
data.head(3)

,postalcode,area,latitude,longitude
0,18906,Straits Boulevard,1.2758,103.8496
1,18907,Straits Boulevard,1.2749,103.8517
2,18910,Marina Gardens Drive,1.2796,103.8690


## Dropping and renaming columns

In [91]:
data.drop([0,3,4,5,6,7,8,11], axis = 1, inplace=True)
print(data.shape)

(121154, 4)


In [92]:
data.rename(columns={1:'postalcode',2:'area',9:'latitude',10:'longitude'}, inplace =True)

## Taking a sample of the original file, cleaning and sorting

The original file contains 121154 instances, this is why I will take a random sample

In [23]:
pcsubset = data.sample(n=320) #Postal code subset of the large database
pcsub = pcsubset[:] #Backing up the data

In [39]:
pcsub.to_csv('sampledata.csv') #Saving the dataframe as CSV in case the kernel dies

<div class="alert alert-block alert-danger">
<b> Use the csv stored file instead:</b> in the line below
</div>

In [334]:
pcsub = pd.read_csv('sampledata.csv') # Reading csv file stored locally

In [335]:
pcsub.drop('Unnamed: 0', axis = 1, inplace=True)

In [336]:
pcsub.sort_values(['area'], ascending=True, axis=0, inplace=True)

In [337]:
pcsub = pcsub.reset_index(drop=True)

In [338]:
pcsub.head()

,postalcode,area,latitude,longitude
0,677851,Almond Street,1.3725,103.7724
1,69878,Amoy Street,1.2801,103.8468
2,544616,Anchorvale Crescent,1.3985,103.8921
3,567920,Ang Mo Kio Avenue 2,1.3774,103.8350
4,561203,Ang Mo Kio Avenue 3,1.3676,103.8446


### Define map coordinates and loop to add markers from random postal codes

In [339]:
# Singapore latitude and longitude values
latitude = 1.290270
longitude = 103.8198

In [340]:
singmap = folium.Map(location=[latitude, longitude], zoom_start=12)

In [341]:
fgroup = folium.map.FeatureGroup() # instantiate a feature group for the incidents in the dataframe

In [342]:
for lat, lng, in zip(pcsub['latitude'], pcsub['longitude']):
    fgroup.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=1, # define how big you want the circle markers to be
            color="black",
            fill=True,
            fill_color="black",
            fill_opacity=0.5
        )
    )

# add incidents to map
singmap.add_child(fgroup)

In [343]:
print(pcsub.shape)
pcsub.head()

(320, 4)


,postalcode,area,latitude,longitude
0,677851,Almond Street,1.3725,103.7724
1,69878,Amoy Street,1.2801,103.8468
2,544616,Anchorvale Crescent,1.3985,103.8921
3,567920,Ang Mo Kio Avenue 2,1.3774,103.8350
4,561203,Ang Mo Kio Avenue 3,1.3676,103.8446


# Using Foursquare to get information from venues

<div class="alert alert-block alert-danger">
<b> Skip the following lines until next red box:</b> Foursquare data is already downloaded (Limited calls)
</div>

In [32]:
CLIENT_ID = 'HJLPKNFYSMPOPHISTM0LKEJHZC1MK2ENDRP1RWT5NAWSYMCN' # your Foursquare ID
CLIENT_SECRET = 'G1I1EM0WQWTCR0JH0H334RF5FJMZMTJGXSLICV01B2P2MMMU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [33]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
snv = getNearbyVenues(names=pcsubset['area'], latitudes=pcsubset['latitude'], longitudes=pcsubset['longitude']) #Singapore nearby venues

Almond Street
Amoy Street
Anchorvale Crescent
Ang Mo Kio Avenue 2
Ang Mo Kio Avenue 3
Ang Mo Kio Avenue 3
Ang Mo Kio Avenue 3
Ash Grove
Ashwood Grove
Barker Road
Bedok Avenue
Bedok Reservoir Road
Bedok Reservoir Road
Bedok Ria Walk
Bedok Road
Belgravia Drive
Belgravia Drive
Bilal Lane
Blandford Drive
Bodmin Drive
Boon Lay Drive
Brookvale Drive
Bukit Merah Central
Burghley Drive
Cairnhill Road
Cambridge Road
Canberra Drive
Cardiff Grove
Cashew Terrace
Casuarina Road
Casuarina Road
Cavan Road
Chancery Hill Walk
Chancery Lane
Changi Road
Charlton Road
Cheng Soon Crescent
Chin Terrace
Chinese Cemetery Path 4
Chiselhurst Grove
Choa Chu Kang Avenue 5
Choa Chu Kang Loop
Choa Chu Kang North 6
Clover Way
Cluny Park
Cluny Road
Compassvale Drive
Coral Island
Corfe Place
Cornwall Gardens
Coronation Road
Corporation Walk
Cuff Road
Cypress Avenue
Da Silva Lane
Dairy Farm Crescent
Deal Road
Dedap Link
Dunbar Walk
Dunlop Street
Dunman Lane
Dunsfold Drive
Duxton Hill
East Coast Road
East Coast Road
Eas

In [37]:
print(snv.shape)
bnv = snv[:]
bnv.head(3)

(17525, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Almond Street,1.3725,103.7724,正味 卤鸭面 Braised Duck,1.376867,103.773162,Noodle House
1,Almond Street,1.3725,103.7724,Chestnut Nature Park,1.371641,103.779792,Nature Preserve
2,Almond Street,1.3725,103.7724,Dairy Farm Nature Park,1.364448,103.776364,Nature Preserve


In [38]:
bnv.to_csv('foursquare.csv') #Saving the dataframe as CSV in case the kernel dies

<div class="alert alert-block alert-danger">
<b> Use the code below:</b> to bring the foursquare data back
</div>

In [344]:
bnv = pd.read_csv('foursquare.csv') # Reading csv file stored locally

## One hot encoding / Preparing data to use in the K-means clustering

In [345]:
ctgrtable = pd.get_dummies(bnv[['Venue Category']], prefix="", prefix_sep="") 
ctgrtable.shape

(17525, 344)

In [346]:
ctgrtable.head(5)

,Accessories Store,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Betting Shop,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cha Chaan Teng,Cheese Shop,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Administrative Building,College Cafeteria,College Gym,College Residence Hall,College Stadium,College Theater,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Light Rail Station,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Lottery Retailer,Lounge,Macanese Restaurant,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,Nature Preserve,Neighborhood,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Racetrack,Ramen Restaurant,Recreation Center,Reservoir,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skate Park,Ski Chalet,Smoke Shop,Sna

In [347]:
ctgrtable.drop("Neighborhood", axis = 1, inplace=True) # Dropping a column called Neighborhood that came directly from the Foursquare database

In [348]:
temptb = pd.concat([bnv["Neighborhood"].reset_index(drop=True),ctgrtable.reset_index(drop=True)], axis=1)
temptb.head()

,Neighborhood,Accessories Store,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Betting Shop,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cha Chaan Teng,Cheese Shop,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Administrative Building,College Cafeteria,College Gym,College Residence Hall,College Stadium,College Theater,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Light Rail Station,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Lottery Retailer,Lounge,Macanese Restaurant,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,Nature Preserve,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Racetrack,Ramen Restaurant,Recreation Center,Reservoir,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skate Park,Ski Chalet,Smoke Shop,Sna

In [349]:
groupt = temptb.groupby('Neighborhood').sum().reset_index()
groupt.head()
print(groupt.shape)

(291, 344)


# The groupt dataframe will be used in a paralel way for generating the top venues list but also to create the final dataframe for the clustering

## 1.- Preparing the dataframe for the k-mean clustering

In [350]:
forcluster = groupt[:] # As a backup but also is the dataframe to be used for clustering purposes

In [351]:
forcluster['latitude'] = forcluster.Neighborhood.map(pcsub.set_index('area')['latitude'].to_dict()) # Bringing the original latitude column

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [352]:
forcluster['longitude'] = forcluster.Neighborhood.map(pcsub.set_index('area')['longitude'].to_dict()) # Bringing the original longitude column

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [353]:
print(forcluster.shape) # This dataframe includes also the latitude and longitude
forcluster.head(2)

(291, 346)


,Neighborhood,Accessories Store,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Betting Shop,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cha Chaan Teng,Cheese Shop,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Administrative Building,College Cafeteria,College Gym,College Residence Hall,College Stadium,College Theater,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Light Rail Station,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Lottery Retailer,Lounge,Macanese Restaurant,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,Nature Preserve,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Racetrack,Ramen Restaurant,Recreation Center,Reservoir,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skate Park,Ski Chalet,Smoke Shop,Sna

## 2.- Top venues per neighborhood

In [354]:
num_top_venues = 10

for hood in groupt['Neighborhood']:
    print("----"+hood+"----")
    temp = groupt[groupt['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Almond Street----
                venue  freq
0     Nature Preserve   3.0
1                Park   3.0
2          Food Court   3.0
3        Noodle House   2.0
4              Market   1.0
5         Coffee Shop   1.0
6  Seafood Restaurant   1.0
7         Flea Market   1.0
8             Parking   1.0
9           Cafeteria   1.0


----Amoy Street----
                 venue  freq
0                Hotel   7.0
1  Japanese Restaurant   6.0
2          Coffee Shop   5.0
3    Korean Restaurant   5.0
4                  Bar   4.0
5           Food Court   4.0
6                 Café   4.0
7           Restaurant   3.0
8               Bakery   3.0
9             Wine Bar   3.0


----Anchorvale Crescent----
                  venue  freq
0  Fast Food Restaurant   4.0
1            Food Court   4.0
2        Cosmetics Shop   3.0
3           Supermarket   2.0
4     Convenience Store   2.0
5           Coffee Shop   2.0
6    Chinese Restaurant   1.0
7  Gym / Fitness Center   1.0
8                   Gym   1.0

In [355]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [356]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
topv = pd.DataFrame(columns=columns)
topv['Neighborhood'] = groupt['Neighborhood']

for ind in np.arange(groupt.shape[0]):
    topv.iloc[ind, 1:] = return_most_common_venues(groupt.iloc[ind, :], num_top_venues)

topv.shape


(291, 11)

In [357]:
topv.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Almond Street,Park,Nature Preserve,Food Court,Noodle House,Seafood Restaurant,Basketball Court,Market,Trail,Parking,Flea Market
1,Amoy Street,Hotel,Japanese Restaurant,Coffee Shop,Korean Restaurant,Café,Food Court,Bar,Restaurant,Bakery,Spanish Restaurant
2,Anchorvale Crescent,Fast Food Restaurant,Food Court,Cosmetics Shop,Convenience Store,Supermarket,Coffee Shop,Bistro,Basketball Court,Gym,Restaurant
3,Ang Mo Kio Avenue 2,Chinese Restaurant,Food Court,Asian Restaurant,Pet Store,Japanese Restaurant,Fast Food Restaurant,Thai Restaurant,Restaurant,Coffee Shop,Pub
4,Ang Mo Kio Avenue 3,Food Court,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Seafood Restaurant,Supermarket,Vegetarian / Vegan Restaurant,Dessert Shop,Noodle House,Japanese Restaurant


In [358]:
topa = topv[:] # Backup

In [359]:
topa.rename(columns={'Neighborhood':'area'}, inplace =True)
print(topa.shape)


(291, 11)


## Running K-Means using the first part

In [360]:
# set number of clusters
kclusters = 10

tgc = forcluster.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tgc)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([6, 3, 6, 9, 4, 2, 6, 2, 6, 0], dtype=int32)

In [361]:
topa.shape

(291, 11)

In [362]:
pcsub.shape

(320, 4)

Let's merge the dataframes that includes the cluster as well as the top 10 venues for each neighborhood.

In [363]:
# add clustering labels
topa.insert(0, 'Cluster Labels', kmeans.labels_)

tablemerged= pcsub[:]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tablemerged = tablemerged.join(topa.set_index('area'), on='area')

In [364]:
tablemerged = tablemerged.drop('postalcode',1)

In [365]:
tablemerged.head(5)

,area,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Almond Street,1.3725,103.7724,6,Park,Nature Preserve,Food Court,Noodle House,Seafood Restaurant,Basketball Court,Market,Trail,Parking,Flea Market
1,Amoy Street,1.2801,103.8468,3,Hotel,Japanese Restaurant,Coffee Shop,Korean Restaurant,Café,Food Court,Bar,Restaurant,Bakery,Spanish Restaurant
2,Anchorvale Crescent,1.3985,103.8921,6,Fast Food Restaurant,Food Court,Cosmetics Shop,Convenience Store,Supermarket,Coffee Shop,Bistro,Basketball Court,Gym,Restaurant
3,Ang Mo Kio Avenue 2,1.3774,103.8350,9,Chinese Restaurant,Food Court,Asian Restaurant,Pet Store,Japanese Restaurant,Fast Food Restaurant,Thai Restaurant,Restaurant,Coffee Shop,Pub
4,Ang Mo Kio Avenue 3,1.3676,103.8446,4,Food Court,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Seafood Restaurant,Supermarket,Vegetarian / Vegan Restaurant,Dessert Shop,Noodle House,Japanese Restaurant


In [391]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tablemerged['latitude'], tablemerged['longitude'], tablemerged['area'], tablemerged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color= 'black',
        weight=1,
        fill=True,
        fill_color= rainbow[cluster -1],
        fill_opacity=1).add_to(map_clusters)

    
    
    
    
map_clusters

### Cluster 0: Mixed restaurants and coffee shops in different areas

In [367]:
tablemerged.loc[tablemerged['Cluster Labels'] == 0, tablemerged.columns[[0] + list(range(4, tablemerged.shape[1]))]]

,area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Bedok Reservoir Road,Food Court,Coffee Shop,Asian Restaurant,Fast Food Restaurant,Supermarket,Park,Indian Restaurant,Chinese Restaurant,Sandwich Place,Café
12,Bedok Reservoir Road,Food Court,Coffee Shop,Asian Restaurant,Fast Food Restaurant,Supermarket,Park,Indian Restaurant,Chinese Restaurant,Sandwich Place,Café
20,Boon Lay Drive,Asian Restaurant,Fast Food Restaurant,Japanese Restaurant,Chinese Restaurant,Food Court,Dessert Shop,Coffee Shop,Convenience Store,Café,Bakery
26,Canberra Drive,Coffee Shop,Fast Food Restaurant,Chinese Restaurant,Asian Restaurant,Thai Restaurant,Indian Restaurant,Food Court,Fried Chicken Joint,Park,Café
34,Changi Road,Chinese Restaurant,Coffee Shop,Seafood Restaurant,Indian Restaurant,Asian Restaurant,BBQ Joint,Noodle House,Food Court,Bar,Train Station
43,Clover Way,Coffee Shop,Chinese Restaurant,Food Court,Seafood Restaurant,Japanese Restaurant,Thai Restaurant,Café,Bubble Tea Shop,Supermarket,Bus Station
69,Enterprise Road,Japanese Restaurant,Asian Restaurant,Coffee Shop,Fast Food Restaurant,Chinese Restaurant,Café,Dessert Shop,Food Court,Fried Chicken Joint,Bubble Tea Shop
71,Faber Green,Food Court,Coffee Shop,Chinese Restaurant,Japanese Restaurant,Shopping Mall,Fried Chicken Joint,Electronics Store,Dim Sum Restaurant,Noodle House,Cosmetics Shop
74,Fernvale Road,Coffee Shop,Food Court,Supermarket,Dessert Shop,Light Rail Station,Asian Restaurant,Bus Station,Indian Restaurant,Café,Shopping Mall
85,Ghim Moh Link,Coffee Shop,Japanese Restaurant,Indian Restaurant,Gym / Fitness Center,Asian Restaurant,Food Court,Chinese Restaurant,Salad Place,Bakery,Shopping Mall


### Cluster 1: Traditional asian restaurants

In [368]:
tablemerged.loc[tablemerged['Cluster Labels'] == 1, tablemerged.columns[[0] + list(range(4, tablemerged.shape[1]))]]

,area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,Charlton Road,Noodle House,Café,Chinese Restaurant,Coffee Shop,Food Court,Asian Restaurant,Dessert Shop,Thai Restaurant,Korean Restaurant,Bakery
60,Dunman Lane,Chinese Restaurant,Noodle House,Asian Restaurant,Café,Indian Restaurant,Food Court,Bar,Italian Restaurant,Vietnamese Restaurant,Japanese Restaurant
63,East Coast Road,Chinese Restaurant,Coffee Shop,Indian Restaurant,Noodle House,Asian Restaurant,Café,Bakery,Dessert Shop,Hotel,Japanese Restaurant
64,East Coast Road,Chinese Restaurant,Coffee Shop,Indian Restaurant,Noodle House,Asian Restaurant,Café,Bakery,Dessert Shop,Hotel,Japanese Restaurant
70,Everitt Road,Asian Restaurant,Chinese Restaurant,Noodle House,Indian Restaurant,Coffee Shop,Hotel,Food Court,Japanese Restaurant,Bakery,BBQ Joint
86,Goodman Road,Chinese Restaurant,Noodle House,Asian Restaurant,Italian Restaurant,Café,Food Court,Ice Cream Shop,Bar,Snack Place,Bakery
87,Goodman Road,Chinese Restaurant,Noodle House,Asian Restaurant,Italian Restaurant,Café,Food Court,Ice Cream Shop,Bar,Snack Place,Bakery
95,Haigsville Drive,Chinese Restaurant,Asian Restaurant,Indian Restaurant,Noodle House,Food Court,Hotel,Malay Restaurant,Japanese Restaurant,Vietnamese Restaurant,Italian Restaurant
109,Jalan Baiduri,Chinese Restaurant,Noodle House,Coffee Shop,Asian Restaurant,Food Court,Bakery,Indian Restaurant,Seafood Restaurant,Café,Bar
147,Joo Chiat Avenue,Chinese Restaurant,Noodle House,Coffee Shop,Asian Restaurant,Food Court,Indian Restaurant,Seafood Restaurant,BBQ Joint,Bar,Bakery


### Cluster 2: Bakeries and cafes

In [369]:
tablemerged.loc[tablemerged['Cluster Labels'] == 2, tablemerged.columns[[0] + list(range(4, tablemerged.shape[1]))]]

,area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Ash Grove,Bakery,Italian Restaurant,Food Court,Pizza Place,Seafood Restaurant,Indian Restaurant,Coffee Shop,Building,Soccer Field,Gym
9,Barker Road,Café,Japanese Restaurant,Chinese Restaurant,Bus Station,Italian Restaurant,Sandwich Place,Fast Food Restaurant,Ramen Restaurant,Food Court,Nightclub
44,Cluny Park,Café,Garden,Bakery,Park,French Restaurant,Food Court,Shopping Mall,Halal Restaurant,Malay Restaurant,Supermarket
45,Cluny Road,Café,Garden,Bakery,Park,Bus Stop,French Restaurant,Shopping Mall,Plaza,Halal Restaurant,Thai Restaurant
50,Coronation Road,Café,Bakery,Shopping Mall,Noodle House,Thai Restaurant,Coffee Shop,Chinese Restaurant,Malay Restaurant,Supermarket,Garden
53,Cypress Avenue,Bus Station,Food Court,Seafood Restaurant,Indian Restaurant,Bakery,Japanese Restaurant,Italian Restaurant,Coffee Shop,Soccer Field,Wine Bar
73,Farrer Road,Café,Garden,Bakery,French Restaurant,Shopping Mall,Noodle House,Metro Station,Food Court,Spanish Restaurant,Malay Restaurant
81,Gardenia Road,Café,Thai Restaurant,Ice Cream Shop,Chinese Restaurant,Spa,Coffee Shop,Trail,French Restaurant,Dessert Shop,Indian Restaurant
91,Greenwood Avenue,Italian Restaurant,Bakery,Pool,Gym,Chinese Restaurant,Japanese Restaurant,Pizza Place,Seafood Restaurant,Restaurant,Tennis Court
92,Greenwood Avenue,Italian Restaurant,Bakery,Pool,Gym,Chinese Restaurant,Japanese Restaurant,Pizza Place,Seafood Restaurant,Restaurant,Tennis Court


### Cluster 3: Business amenities 

In [370]:
tablemerged.loc[tablemerged['Cluster Labels'] == 3, tablemerged.columns[[0] + list(range(4, tablemerged.shape[1]))]]

,area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Amoy Street,Hotel,Japanese Restaurant,Coffee Shop,Korean Restaurant,Café,Food Court,Bar,Restaurant,Bakery,Spanish Restaurant
32,Chancery Hill Walk,Café,Chinese Restaurant,Hotel,Bakery,Coffee Shop,Food Court,Japanese Restaurant,Supermarket,Ramen Restaurant,Italian Restaurant
33,Chancery Lane,Chinese Restaurant,Café,Coffee Shop,Hotel,Bakery,Food Court,Japanese Restaurant,Italian Restaurant,Asian Restaurant,Sandwich Place
62,Duxton Hill,Japanese Restaurant,Coffee Shop,Bakery,Cocktail Bar,Hotel,Italian Restaurant,Bar,Café,Gym / Fitness Center,Ramen Restaurant
82,Gentle Drive,Chinese Restaurant,Hotel,Café,Bakery,Coffee Shop,Japanese Restaurant,Ramen Restaurant,Food Court,Italian Restaurant,Dessert Shop
97,Havelock Road,Chinese Restaurant,Japanese Restaurant,Café,Noodle House,Coffee Shop,Hotel,Food Court,Bakery,Bookstore,Park
104,Holland Rise,Bakery,Japanese Restaurant,Bar,Chinese Restaurant,Ice Cream Shop,Food Court,Shopping Mall,Café,Indian Restaurant,Coffee Shop
128,Jalan Novena,Chinese Restaurant,Hotel,Café,Bakery,Food Court,Coffee Shop,Dessert Shop,Japanese Restaurant,Asian Restaurant,Supermarket
143,Jalan Tiga Ratus,Café,Coffee Shop,Fast Food Restaurant,Chinese Restaurant,Japanese Restaurant,Sandwich Place,Food Court,Sporting Goods Shop,Indian Restaurant,Fried Chicken Joint
210,Nassim Road,Hotel,Café,Park,Chinese Restaurant,Japanese Restaurant,Spa,Bar,Garden,Thai Restaurant,Italian Restaurant


### Cluster 4: Popular food, hawker centers and Asian restaurants

In [371]:
tablemerged.loc[tablemerged['Cluster Labels'] == 4, tablemerged.columns[[0] + list(range(4, tablemerged.shape[1]))]]

,area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Ang Mo Kio Avenue 3,Food Court,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Seafood Restaurant,Supermarket,Vegetarian / Vegan Restaurant,Dessert Shop,Noodle House,Japanese Restaurant
5,Ang Mo Kio Avenue 3,Food Court,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Seafood Restaurant,Supermarket,Vegetarian / Vegan Restaurant,Dessert Shop,Noodle House,Japanese Restaurant
6,Ang Mo Kio Avenue 3,Food Court,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Seafood Restaurant,Supermarket,Vegetarian / Vegan Restaurant,Dessert Shop,Noodle House,Japanese Restaurant
83,Geylang Road,Chinese Restaurant,Noodle House,Food Court,Seafood Restaurant,Hotel,Dim Sum Restaurant,Asian Restaurant,BBQ Joint,Vegetarian / Vegan Restaurant,Indian Restaurant
84,Geylang Road,Chinese Restaurant,Noodle House,Food Court,Seafood Restaurant,Hotel,Dim Sum Restaurant,Asian Restaurant,BBQ Joint,Vegetarian / Vegan Restaurant,Indian Restaurant
99,Highland Road,Chinese Restaurant,Noodle House,Food Court,Café,Coffee Shop,Asian Restaurant,Bus Station,Park,Vegetarian / Vegan Restaurant,Snack Place
100,Highland Road,Chinese Restaurant,Noodle House,Food Court,Café,Coffee Shop,Asian Restaurant,Bus Station,Park,Vegetarian / Vegan Restaurant,Snack Place
117,Jalan Hock Chye,Chinese Restaurant,Noodle House,Coffee Shop,Food Court,Café,Asian Restaurant,Bus Station,Dessert Shop,Bakery,Snack Place
118,Jalan Hock Chye,Chinese Restaurant,Noodle House,Coffee Shop,Food Court,Café,Asian Restaurant,Bus Station,Dessert Shop,Bakery,Snack Place
145,Jansen Road,Chinese Restaurant,Asian Restaurant,Coffee Shop,Noodle House,Café,Food Court,Park,Bus Station,Snack Place,Bus Line


### Cluster 5: Indian area and restaurants

In [372]:
tablemerged.loc[tablemerged['Cluster Labels'] == 5, tablemerged.columns[[0] + list(range(4, tablemerged.shape[1]))]]

,area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Cambridge Road,Indian Restaurant,Chinese Restaurant,Café,Hotel,Coffee Shop,Food Court,Restaurant,Seafood Restaurant,Asian Restaurant,Japanese Restaurant
52,Cuff Road,Indian Restaurant,Café,Chinese Restaurant,Bakery,Hotel,Vegetarian / Vegan Restaurant,Asian Restaurant,Dessert Shop,Dim Sum Restaurant,BBQ Joint
59,Dunlop Street,Indian Restaurant,Café,Chinese Restaurant,Hotel,Vegetarian / Vegan Restaurant,Bakery,Restaurant,Asian Restaurant,Hostel,Dessert Shop
111,Jalan Besar,Indian Restaurant,Chinese Restaurant,Café,Hostel,BBQ Joint,Hotel,Restaurant,Vegetarian / Vegan Restaurant,Bakery,Supermarket
235,Race Course Road,Indian Restaurant,Chinese Restaurant,Café,Hotel,Dessert Shop,Hostel,Food Court,Noodle House,Soup Place,Vegetarian / Vegan Restaurant
249,Serangoon Road,Indian Restaurant,Chinese Restaurant,Café,Hotel,Vegetarian / Vegan Restaurant,Hostel,BBQ Joint,Asian Restaurant,Dim Sum Restaurant,Dessert Shop
299,Upper Dickson Road,Indian Restaurant,Café,Chinese Restaurant,Hotel,Vegetarian / Vegan Restaurant,Bakery,Restaurant,Asian Restaurant,Hostel,Dessert Shop


### Cluster 6: Entertainment, leisure, parks, utilities and food

In [373]:
tablemerged.loc[tablemerged['Cluster Labels'] == 6, tablemerged.columns[[0] + list(range(4, tablemerged.shape[1]))]]

,area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Almond Street,Park,Nature Preserve,Food Court,Noodle House,Seafood Restaurant,Basketball Court,Market,Trail,Parking,Flea Market
2,Anchorvale Crescent,Fast Food Restaurant,Food Court,Cosmetics Shop,Convenience Store,Supermarket,Coffee Shop,Bistro,Basketball Court,Gym,Restaurant
8,Ashwood Grove,Fast Food Restaurant,Food Court,Pizza Place,Grocery Store,Shopping Mall,Pool,Supermarket,Gym,Hainan Restaurant,Halal Restaurant
10,Bedok Avenue,Beach,Campground,Resort,Food Court,Grocery Store,School,Bike Rental / Bike Share,Malay Restaurant,Noodle House,Supermarket
13,Bedok Ria Walk,Noodle House,Dessert Shop,Malay Restaurant,Coffee Shop,Café,Asian Restaurant,Food Court,Steakhouse,Bus Stop,Supermarket
14,Bedok Road,Asian Restaurant,Noodle House,Chinese Restaurant,BBQ Joint,Seafood Restaurant,Malay Restaurant,Dessert Shop,Coffee Shop,Shopping Mall,Café
15,Belgravia Drive,Café,Bus Stop,Thai Restaurant,Asian Restaurant,Bakery,Electronics Store,Diner,Shopping Mall,Video Store,Dessert Shop
16,Belgravia Drive,Café,Bus Stop,Thai Restaurant,Asian Restaurant,Bakery,Electronics Store,Diner,Shopping Mall,Video Store,Dessert Shop
17,Bilal Lane,Bus Line,Coffee Shop,Noodle House,Soccer Field,Flower Shop,Train Station,Bus Stop,Bus Station,Garden,Malay Restaurant
21,Brookvale Drive,Café,Bakery,College Cafeteria,Grocery Store,Military Base,Bus Station,Sandwich Place,Theater,Canal,Supermarket


### Cluster 7: Upscale venues

In [374]:
tablemerged.loc[tablemerged['Cluster Labels'] == 7, tablemerged.columns[[0] + list(range(4, tablemerged.shape[1]))]]

,area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Cairnhill Road,Hotel,Boutique,Japanese Restaurant,Chinese Restaurant,Bakery,Sushi Restaurant,Shopping Mall,Bubble Tea Shop,Cosmetics Shop,Supermarket
67,Emerald Hill Road,Japanese Restaurant,Hotel,Boutique,Shopping Mall,Clothing Store,Sushi Restaurant,Chinese Restaurant,Wine Bar,Bubble Tea Shop,Korean Restaurant
114,Jalan Elok,Boutique,Hotel,Japanese Restaurant,Sushi Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Shopping Mall,Cosmetics Shop,Tea Room
164,Leonie Hill,Boutique,Shopping Mall,Japanese Restaurant,Bubble Tea Shop,Hotel,Chinese Restaurant,Coffee Shop,Cosmetics Shop,Sushi Restaurant,Clothing Store


### Cluster 8: Coffee shops and traditional restaurants in residential areas

In [375]:
tablemerged.loc[tablemerged['Cluster Labels'] == 8, tablemerged.columns[[0] + list(range(4, tablemerged.shape[1]))]]

,area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Cavan Road,Café,Chinese Restaurant,Coffee Shop,Indian Restaurant,Hostel,Hotel,BBQ Joint,Noodle House,Restaurant,Food Court
58,Dunbar Walk,Coffee Shop,Chinese Restaurant,Indian Restaurant,Noodle House,Asian Restaurant,Food Court,Gastropub,Italian Restaurant,Bar,BBQ Joint
75,Fidelio Street,Coffee Shop,Indian Restaurant,Chinese Restaurant,Supermarket,Dessert Shop,Café,Asian Restaurant,Bus Station,Gastropub,Food Court
78,Fourth Street,Coffee Shop,Indian Restaurant,Chinese Restaurant,Café,Dessert Shop,Gastropub,Pizza Place,Seafood Restaurant,Supermarket,Spa
79,Frankel Street,Chinese Restaurant,Coffee Shop,Indian Restaurant,Noodle House,Asian Restaurant,Food Court,Seafood Restaurant,Café,BBQ Joint,Italian Restaurant
121,Jalan Keris,Coffee Shop,Noodle House,Café,Chinese Restaurant,Dessert Shop,Bus Station,Park,Asian Restaurant,Indian Restaurant,Beach
122,Jalan Keris,Coffee Shop,Noodle House,Café,Chinese Restaurant,Dessert Shop,Bus Station,Park,Asian Restaurant,Indian Restaurant,Beach
142,Jalan Terang Bulan,Coffee Shop,Chinese Restaurant,Café,Asian Restaurant,Supermarket,Food Court,Bus Station,Bakery,Fast Food Restaurant,Indian Restaurant
144,Jalan Yasin,Chinese Restaurant,Coffee Shop,Indian Restaurant,Food Court,Noodle House,Seafood Restaurant,Asian Restaurant,BBQ Joint,Café,Food Truck
160,Lavender Street,Chinese Restaurant,Café,Hotel,Indian Restaurant,Coffee Shop,Noodle House,BBQ Joint,Hostel,Dessert Shop,Restaurant


### Cluster 9: Food around Serangoon

In [376]:
tablemerged.loc[tablemerged['Cluster Labels'] == 9, tablemerged.columns[[0] + list(range(4, tablemerged.shape[1]))]]

,area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Ang Mo Kio Avenue 2,Chinese Restaurant,Food Court,Asian Restaurant,Pet Store,Japanese Restaurant,Fast Food Restaurant,Thai Restaurant,Restaurant,Coffee Shop,Pub
18,Blandford Drive,Chinese Restaurant,Food Court,Bus Station,Park,Bakery,Indonesian Restaurant,Café,Korean Restaurant,Noodle House,Asian Restaurant
19,Bodmin Drive,Chinese Restaurant,Food Court,Park,Bakery,Noodle House,Bus Station,Korean Restaurant,Café,Asian Restaurant,Italian Restaurant
22,Bukit Merah Central,Chinese Restaurant,Coffee Shop,Food Court,Bus Station,Gym,Clothing Store,Bakery,Fast Food Restaurant,Trail,Noodle House
23,Burghley Drive,Chinese Restaurant,Food Court,Bus Station,Park,Bakery,Noodle House,Café,Korean Restaurant,Asian Restaurant,Sushi Restaurant
27,Cardiff Grove,Chinese Restaurant,Japanese Restaurant,Coffee Shop,Bus Station,Asian Restaurant,Ice Cream Shop,Bakery,Noodle House,Clothing Store,Indonesian Restaurant
29,Casuarina Road,Bus Station,Chinese Restaurant,Food Court,Restaurant,Japanese Restaurant,Asian Restaurant,Coffee Shop,Park,Pet Store,Café
30,Casuarina Road,Bus Station,Chinese Restaurant,Food Court,Restaurant,Japanese Restaurant,Asian Restaurant,Coffee Shop,Park,Pet Store,Café
36,Cheng Soon Crescent,Korean Restaurant,Chinese Restaurant,Café,Food Court,College Cafeteria,Thai Restaurant,Coffee Shop,Bakery,Massage Studio,Seafood Restaurant
39,Chiselhurst Grove,Chinese Restaurant,Park,Coffee Shop,Pub,Bus Station,Café,Asian Restaurant,Bakery,Noodle House,Food Court


In [377]:
tablemerged

,area,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Almond Street,1.3725,103.7724,6,Park,Nature Preserve,Food Court,Noodle House,Seafood Restaurant,Basketball Court,Market,Trail,Parking,Flea Market
1,Amoy Street,1.2801,103.8468,3,Hotel,Japanese Restaurant,Coffee Shop,Korean Restaurant,Café,Food Court,Bar,Restaurant,Bakery,Spanish Restaurant
2,Anchorvale Crescent,1.3985,103.8921,6,Fast Food Restaurant,Food Court,Cosmetics Shop,Convenience Store,Supermarket,Coffee Shop,Bistro,Basketball Court,Gym,Restaurant
3,Ang Mo Kio Avenue 2,1.3774,103.8350,9,Chinese Restaurant,Food Court,Asian Restaurant,Pet Store,Japanese Restaurant,Fast Food Restaurant,Thai Restaurant,Restaurant,Coffee Shop,Pub
4,Ang Mo Kio Avenue 3,1.3676,103.8446,4,Food Court,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Seafood Restaurant,Supermarket,Vegetarian / Vegan Restaurant,Dessert Shop,Noodle House,Japanese Restaurant
5,Ang Mo Kio Avenue 3,1.3708,103.8435,4,Food Court,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Seafood Restaurant,Supermarket,Vegetarian / Vegan Restaurant,Dessert Shop,Noodle House,Japanese Restaurant
6,Ang Mo Kio Avenue 3,1.3699,103.8421,4,Food Court,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Seafood Restaurant,Supermarket,Vegetarian / Vegan Restaurant,Dessert Shop,Noodle House,Japanese Restaurant
7,Ash Grove,1.3329,103.8030,2,Bakery,Italian Restaurant,Food Court,Pizza Place,Seafood Restaurant,Indian Restaurant,Coffee Shop,Building,Soccer Field,Gym
8,Ashwood Grove,1.4288,103.7767,6,Fast Food Restaurant,Food Court,Pizza Place,Grocery Store,Shopping Mall,Pool,Supermarket,Gym,Hainan Restaurant,Halal Restaurant
9,Barker Road,1.3214,103.8351,2,Café,Japanese Restaurant,Chinese Restaurant,Bus Station,Italian Restaurant,Sandwich Place,Fast Food Restaurant,Ramen Restaurant,Food Court,Nightclub


## Plotting recommended cluster

In [392]:
folium.Circle(location=[1.3282, 103.8083], radius=2000, 
        popup='Laurelhurst Park', line_color='#3186cc',
        fill_color='#3186cc', fill=True).add_to(map_clusters)

In [393]:
map_clusters